In [344]:
using JuMP
using Ipopt
using CSV, DataFrames
using PyPlot, LaTeXStrings, PyCall
using Interpolations, Distributions, Dierckx

In [345]:
ρ_air = 1.225
L_pp = 7.0
B = 1.27  # 船幅[m]
d = 0.4
D = 0.6563 # 深さ[m]
A_OD = 0.65 # デッキ上の構造物の側面投影面積[m^2]
H_BR = 0.85 # 喫水からブリッジ主要構造物の最高位[m]
H_C = 0.235 # 喫水から側面積中心までの高さ[m]
C = 0.0 # 船体中心から側面積中心までの前後方向座標(船首方向を正)[m]

A_OD = A_OD # デッキ上の構造物の側面投影面積[m^2]
A_F = (D - d) * B  # 船体の正面投影面積[m^2]
A_L = (D - d) * L_pp

1.7940999999999998

In [346]:
function wind_force_and_moment_coefficients(
    ψ_A,
    L_pp,
    B,
    A_OD,
    A_F,
    A_L,
    H_BR,
    H_C,
    C,
)
    #C_LF1の場合で調整
    C_CF = 0.404 + 0.368 * A_F / (B * H_BR) + 0.902 * H_BR / L_pp

    if deg2rad(0) <= ψ_A <= deg2rad(90)
        C_LF = -0.992 + 0.507 * A_L / (L_pp * B) + 1.162 * C / L_pp
        C_XLI = 0.458 + 3.245 * A_L / (L_pp * H_BR) - 2.313 * A_F / (B * H_BR)
        C_ALF = -0.585 - 0.906 * A_OD / A_L + 3.239 * B / L_pp
        C_YLI = pi * A_L / L_pp^2 + 0.116 + 3.345 * A_F / (L_pp * B)

        C_X = C_LF * cos(ψ_A) + C_XLI * (sin(ψ_A) - sin(ψ_A) * cos(ψ_A)^2 / 2) * sin(ψ_A) * cos(ψ_A) + C_ALF * sin(ψ_A) * cos(ψ_A)^3
        C_Y = C_CF * sin(ψ_A)^2 + C_YLI * (cos(ψ_A) + sin(ψ_A)^2 * cos(ψ_A) / 2) * sin(ψ_A) * cos(ψ_A)
        C_N = C_Y * (0.297 * C / L_pp - 0.149 * (ψ_A - deg2rad(90)))

    elseif deg2rad(90) < ψ_A <= deg2rad(180)
        C_LF =
            0.018 - 5.091 * B / L_pp + 10.367 * H_C / L_pp - 3.011 * A_OD / L_pp^2 -
            0.341 * A_F / B^2
        C_XLI =
            -1.901 + 12.727 * A_L / (L_pp * H_BR) + 24.407 * A_F / A_L -
            40.310 * B / L_pp - 0.341 * A_F / (B * H_BR)
        C_ALF = -0.314 - 1.117 * A_OD / A_L
        C_YLI = pi * A_L / L_pp^2 + 0.446 + 2.192 * A_F / L_pp^2

        C_X = C_LF * cos(ψ_A) + C_XLI * (sin(ψ_A) - sin(ψ_A) * cos(ψ_A)^2 / 2) * sin(ψ_A) * cos(ψ_A) + C_ALF * sin(ψ_A) * cos(ψ_A)^3
        C_Y = C_CF * sin(ψ_A)^2 + C_YLI * (cos(ψ_A) + sin(ψ_A)^2 * cos(ψ_A) / 2) * sin(ψ_A) * cos(ψ_A)
        C_N = C_Y * (0.297 * C / L_pp - 0.149 * (ψ_A - deg2rad(90)))

    elseif deg2rad(180) < ψ_A <= deg2rad(270)
        C_LF =
            0.018 - 5.091 * B / L_pp + 10.367 * H_C / L_pp - 3.011 * A_OD / L_pp^2 -
            0.341 * A_F / B^2
        C_XLI =
            -1.901 + 12.727 * A_L / (L_pp * H_BR) + 24.407 * A_F / A_L -
            40.310 * B / L_pp - 0.341 * A_F / (B * H_BR)
        C_ALF = -0.314 - 1.117 * A_OD / A_L
        C_YLI = pi * A_L / L_pp^2 + 0.446 + 2.192 * A_F / L_pp^2
        ψ_A = 2*pi - ψ_A
        C_X = (C_LF * cos(ψ_A) + C_XLI * (sin(ψ_A) - sin(ψ_A) * cos(ψ_A)^2 / 2) * sin(ψ_A) * cos(ψ_A) + C_ALF * sin(ψ_A) * cos(ψ_A)^3)
        C_Y = -(C_CF * sin(ψ_A)^2 + C_YLI * (cos(ψ_A) + sin(ψ_A)^2 * cos(ψ_A) / 2) * sin(ψ_A) * cos(ψ_A))
        C_N = (C_Y * (0.297 * C / L_pp - 0.149 * (ψ_A - deg2rad(90))))
    
    elseif deg2rad(270) < ψ_A <= deg2rad(360)
        C_LF = -0.992 + 0.507 * A_L / (L_pp * B) + 1.162 * C / L_pp
        C_XLI = 0.458 + 3.245 * A_L / (L_pp * H_BR) - 2.313 * A_F / (B * H_BR)
        C_ALF = -0.585 - 0.906 * A_OD / A_L + 3.239 * B / L_pp
        C_YLI = pi * A_L / L_pp^2 + 0.116 + 3.345 * A_F / (L_pp * B)

        ψ_A = 2*pi - ψ_A
        C_X = (C_LF * cos(ψ_A) + C_XLI * (sin(ψ_A) - sin(ψ_A) * cos(ψ_A)^2 / 2) * sin(ψ_A) * cos(ψ_A) + C_ALF * sin(ψ_A) * cos(ψ_A)^3)
        C_Y = -(C_CF * sin(ψ_A)^2 + C_YLI * (cos(ψ_A) + sin(ψ_A)^2 * cos(ψ_A) / 2) * sin(ψ_A) * cos(ψ_A))
        C_N = (C_Y * (0.297 * C / L_pp - 0.149 * (ψ_A - deg2rad(90))))
    end
    
    C_X, C_Y, C_N
end

ψ_A_vec = deg2rad.(collect(0:10:360))
C_X_vec = Array{Float64}(undef, length(ψ_A_vec))
C_Y_vec = Array{Float64}(undef, length(ψ_A_vec))
C_N_vec = Array{Float64}(undef, length(ψ_A_vec))
for (index, ψ_A) in enumerate(ψ_A_vec)
    C_X, C_Y, C_N = wind_force_and_moment_coefficients(
        ψ_A,
        L_pp,
        B,
        A_OD,
        A_F,
        A_L,
        H_BR,
        H_C,
        C,
    )
    C_X_vec[index] = C_X
    C_Y_vec[index] = C_Y
    C_N_vec[index] = C_N
end
spl_C_X = Spline1D(ψ_A_vec, C_X_vec)
spl_C_Y = Spline1D(ψ_A_vec, C_Y_vec)
spl_C_N = Spline1D(ψ_A_vec, C_N_vec)

# function approx_CX(ψ)
#     return spl_C_X(ψ)
# end
# function approx_CY(ψ)
#     return spl_C_Y(ψ)
# end
# function approx_CN(ψ)
#     return spl_C_N(ψ) 
# end

ψ_A_discrete = collect(ψ_A_vec)
C_X_discrete = collect(C_X_vec)
C_Y_discrete = collect(C_Y_vec)
C_N_discrete = collect(C_N_vec)

function approx_CX(ψ)
    index = findfirst(x -> x >= ψ, ψ_A_discrete)
    return C_X_discrete[index]
end
function approx_CY(ψ)
    index = findfirst(x -> x >= ψ, ψ_A_discrete)
    return C_Y_discrete[index]
end
function approx_CN(ψ)
    index = findfirst(x -> x >= ψ, ψ_A_discrete)
    return C_N_discrete[index]
end

approx_CN (generic function with 1 method)

In [347]:
function wind(u, v, psi, X_F, Y_F, N_F)
    model = Model(Ipopt.Optimizer)

    register(model, :mod, 2, mod; autodiff = true)
    register(model, :approx_CX, 1, approx_CX; autodiff = true)
    register(model, :approx_CY, 1, approx_CY; autodiff = true)
    register(model, :approx_CN, 1, approx_CN; autodiff = true)

    @variable(model, 0.0 <= U_wind <= 20.0, start = 10.0)
    @variable(model, 0.0*pi <= psi_wind <= 2.0*pi, start = pi)

    @variable(model, u_A, start = u)
    @variable(model, v_A, start = v)
    @variable(model, U_A, start = sqrt((u)^2 + (v)^2))
    @variable(model, ψ_A, start = psi)
    @variable(model, w1, start = 0.0001)
    @variable(model, w2, start = 0.0001)
    @variable(model, w3, start = 0.0001)

    @NLconstraint(model, u_A == u + U_wind * cos(psi_wind - psi))
    @NLconstraint(model, v_A == v + U_wind * sin(psi_wind - psi))
    @NLconstraint(model, U_A == sqrt(u_A^2 + v_A^2))
    # @constraint(model, ψ_A == mod(atan(v_A, u_A), 2*pi))
    @NLconstraint(model, ψ_A == -atan(v_A, u_A))
    @NLconstraint(model, w1 == X_F - ρ_air * A_F * approx_CX(ψ_A) / 2 * U_A^2)
    @NLconstraint(model, w2 == Y_F - ρ_air * A_L * approx_CY(ψ_A) / 2 * U_A^2)
    @NLconstraint(model, w3 == N_F - ρ_air * A_L * L_pp * approx_CN(ψ_A) / 2 * U_A^2)

    @NLobjective(model, Min, w1^2 + w2^2 + w3^2)

    optimize!(model)

    return value(U_wind), value(psi_wind), value(w1), value(w2), value(w3), value(ψ_A)
end

wind (generic function with 1 method)

In [348]:
data = CSV.read("/home/hirukawa/myenv/Lab/MCMC-MPC_KVLCC2/MPC/observation/obs_10deg_10ms_EF.csv", DataFrame)
dt = 100
u = data[dt, "u"]
v = data[dt, "v"]
psi = data[dt, "ψ"]
X_F = data[dt, "X_wind"]
Y_F = data[dt, "Y_wind"]  
N_F = data[dt, "N_wind"]
println(u)
a, b, c, d, e, f = wind(u, v, psi, X_F, Y_F, N_F)

1.4308672882191036
This is Ipopt version 3.14.14, running with linear solver MUMPS 5.6.2.

Number of nonzeros in equality constraint Jacobian...:       21
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:        0

Total number of variables............................:        9
                     variables with only lower bounds:        0
                variables with lower and upper bounds:        2
                     variables with only upper bounds:        0
Total number of equality constraints.................:        7
Total number of inequality constraints...............:        0
        inequality constraints with only lower bounds:        0
   inequality constraints with lower and upper bounds:        0
        inequality constraints with only upper bounds:        0

iter    objective    inf_pr   inf_du lg(mu)  ||d||  lg(rg) alpha_du alpha_pr  ls
   0  3.0000000e-08 4.95e+01 1.33e-04   0.0 0.00e+00    - 

(3.499730630721904, 4.378300094984814, -2.601194761050289e-12, -48.04953859867125, -6.820566388168301, -2.7334204060357874)

In [350]:
println("U_wind = ", a)
println("psi_wind = ", b)
println("w1 = ", c)
println("w2 = ", d)
println("w3 = ", e)
println("U_A = ", f)

U_wind = 3.499730630721904
psi_wind = 4.378300094984814
w1 = -2.601194761050289e-12
w2 = -48.04953859867125
w3 = -6.820566388168301
U_A = -2.7334204060357874
